In [13]:
import numpy as np 
from tslearn.datasets import UCR_UEA_datasets
from tslearn.svm import TimeSeriesSVC
from tslearn.preprocessing import TimeSeriesScalerMinMax

# Parameters

In [14]:
dataset='BasicMotions'

# Load Data

In [15]:
train_x,train_y, test_x, test_y=UCR_UEA_datasets().load_dataset(dataset)
train_x = TimeSeriesScalerMinMax().fit_transform(train_x)
test_x = TimeSeriesScalerMinMax().fit_transform(test_x)

In [16]:
print(train_x.shape)
print(test_x.shape)

(40, 100, 6)
(40, 100, 6)


In [17]:
model = TimeSeriesSVC(kernel="gak", gamma="auto", probability=True)
model.fit(train_x, train_y)
print("Correct classification rate:", model.score(test_x, test_y))

Correct classification rate: 0.925


# Explanation Algo

In [18]:
item=test_x[10].reshape(1,test_x.shape[1],test_x.shape[2])
shape=item.shape
y_target= model.predict_proba(item)
pred_y=  model.predict_proba(train_x)

In [19]:
y_target

array([[0.02670258, 0.74590771, 0.1267004 , 0.10068931]])

In [20]:
np.argmax(pred_y,axis=1)

array([2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 3, 2, 2, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [21]:
from TSInterpret.InterpretabilityModels.counterfactual.Ates import AtesCF

exp_model= AtesCF(model,(train_x,np.argmax(pred_y,axis=1)),backend='SK', mode='time')


In [22]:
#TODO Why does Brute work with same number and not with zero
#TODO Opt with 0 and 1? 
#TODO Opt takes longer than brut 
np.argmax(y_target)
exp = exp_model.explain(item,2, method= 'opt')

Run Brute Force as Backup.
Best (1, 6, 100)
Other (1,)
Target [1]


In [23]:
array, label=exp

In [24]:
org_label=np.argmax(y_target)
cf_label=label[0]
exp=array

exp_model.plot(item,org_label,exp,cf_label,figsize=(15,15))

Items are identical
